In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle
from itertools import combinations

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# R
# G
# H

category = 'R'
# category = 'G'
# category = 'H'

In [3]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

13415 races loaded!


In [4]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

4467 R races!


In [5]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
1059,1610,RANDWICK,NSW,S,R,2017-09-16,1,TAB HIGHWAY HANDICAP,2017-09-16 02:15:00,Paying,1000,"[[4], [10], [2], [5]]",11.0,"[{""runnerName"": ""AZZIE'S READY"", ""runnerNumber...",NaN,NaN,NaN,NaN,"[[4], [10], [2], [5]]","[{'runnerName': 'AZZIE'S READY', 'runnerNumber..."
1060,1611,RANDWICK,NSW,S,R,2017-09-16,2,SHOOT OUT MILE,2017-09-16 02:50:00,Paying,1600,"[[3], [5], [8], [7]]",10.0,"[{""runnerName"": ""GOLD HORIZON"", ""runnerNumber""...",NaN,NaN,NaN,NaN,"[[3], [5], [8], [7]]","[{'runnerName': 'GOLD HORIZON', 'runnerNumber'..."
1061,1612,RANDWICK,NSW,S,R,2017-09-16,3,SCHWEPPES HERITAGE STAKES,2017-09-16 03:25:00,Paying,1100,"[[2], [1], [8], [6]]",10.0,"[{""runnerName"": ""LUCKY LOUIE"", ""runnerNumber"":...",NaN,NaN,NaN,NaN,"[[2], [1], [8], [6]]","[{'runnerName': 'LUCKY LOUIE', 'runnerNumber':..."
1062,1613,RANDWICK,NSW,S,R,2017-09-16,4,BILL RITCHIE HANDICAP,2017-09-16 04:00:00,Paying,1400,"[[7], [3], [4], [6]]",9.0,"[{""runnerName"": ""PAJARO"", ""runnerNumber"": 6, ""...",NaN,NaN,NaN,NaN,"[[7], [3], [4], [6]]","[{'runnerName': 'PAJARO', 'runnerNumber': 6, '..."
1063,1614,RANDWICK,NSW,S,R,2017-09-16,5,MCGRATH ESTATE TEA ROSE STAKES,2017-09-16 04:35:00,Paying,1400,"[[6], [2], [5], [1]]",12.0,"[{""runnerName"": ""PANDEMONIUM"", ""runnerNumber"":...",NaN,NaN,NaN,NaN,"[[6], [2], [5], [1]]","[{'runnerName': 'PANDEMONIUM', 'runnerNumber':..."


In [8]:
# create quinella pairs
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
#     print(race)

    num_runners = len([r for r in race['runners'] if r['win_odds'] and r['place_odds']])

    # add num_runners to all runners
    for runner in race['runners']:
#         print(runner)
        runner['num_runners'] = num_runners
        runner['race_type'] = race['race_type']
        runner['quinella'] = race['quinella']

    data_all = data_all.append(race['runners'])

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
      ...  
7      True
8      True
9      True
10     True
0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
Name: quinella, Length: 48079, dtype: bool

In [11]:
data_pre.tail(5)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,num_runners,parimutuel,...,quinella,race_type,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
10,NaN,NaN,NaN,NaN,9,-1.0,0,"{'returnWin': 12, 'returnWinOpen': 21, 'return...",15,"{'bettingStatus': 'Normal', 'returnWin': 17.3,...",...,0.0,R,M GRIFFITHS,MOLE TRAP,11,MISS NICKY MARTIN,12.0,0.083333,7,0.060122
11,NaN,NaN,NaN,NaN,10,-1.0,0,"{'returnWin': 34, 'returnWinOpen': 34, 'return...",15,"{'bettingStatus': 'Normal', 'returnWin': 41.5,...",...,0.0,R,L TREADWELL,SUNDANCE BOY,12,G FIERRO,34.0,0.029412,12,0.021219
12,NaN,NaN,NaN,NaN,11,-1.0,3,"{'returnWin': 7.5, 'returnWinOpen': 7.5, 'retu...",15,"{'bettingStatus': 'Normal', 'returnWin': 7.5, ...",...,0.0,R,H SKELTON,COME ON LOUIS,13,OLIVER GREENALL,7.5,0.133333,4,0.096195
13,NaN,NaN,NaN,NaN,12,-1.0,2,"{'returnWin': 13, 'returnWinOpen': 13, 'return...",15,"{'bettingStatus': 'Normal', 'returnWin': 14.3,...",...,0.0,R,B HUGHES,LADY VIVONA,14,MISS L HARRISON,13.0,0.076923,9,0.055497
14,NaN,NaN,NaN,NaN,13,-1.0,0,"{'returnWin': 34, 'returnWinOpen': 41, 'return...",15,"{'bettingStatus': 'Normal', 'returnWin': 59.3,...",...,0.0,R,A NICOL,GASOLINE,15,M E SOWERSBY,34.0,0.029412,12,0.021219


In [10]:
# drop scratched
data = data_pre.dropna(subset=['win_odds', 'place_odds', 'quinella'])
data = data[(data.win_odds > 0)]
data = data[(data.quinella > 0)]
data.describe()

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,num_runners,place_odds,quinella,runnerNumber,win_odds,win_perc,win_rank,win_scaled
count,0.0,0.0,0.0,0.0,4352.000000,4352.000000,4352.000000,4352.000000,4352.000000,4352.000000,4352.000000,4352.000000,4352.000000,4352.000000,4352.000000
mean,NaN,NaN,NaN,NaN,5.660156,-0.303194,0.875919,10.710018,4.611121,1538.811857,6.173254,22.310593,0.130889,5.730928,0.102711
std,NaN,NaN,NaN,NaN,3.430780,1.301987,1.307563,3.232791,4.679809,3321.481839,3.803353,32.129238,0.132026,3.548201,0.101772
min,NaN,NaN,NaN,NaN,1.000000,-1.000000,0.000000,3.000000,0.000000,1.200000,1.000000,1.010000,0.000000,1.000000,0.000000
25%,NaN,NaN,NaN,NaN,3.000000,-1.000000,0.000000,8.000000,2.000000,138.000000,3.000000,5.500000,0.038462,3.000000,0.031507
50%,NaN,NaN,NaN,NaN,5.000000,-1.000000,0.000000,10.000000,3.050000,381.300000,6.000000,11.000000,0.090909,5.000000,0.069544
75%,NaN,NaN,NaN,NaN,8.000000,0.000000,2.000000,12.000000,5.300000,1336.900000,9.000000,23.750000,0.181818,8.000000,0.140187
max,NaN,NaN,NaN,NaN,21.000000,4.500000,4.000000,21.000000,65.200000,33751.900000,21.000000,401.000000,0.990099,21.000000,0.794224


In [ ]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

In [ ]:
# xp = fixed odds perc
xp = data['win_perc']
xp.describe()

In [ ]:
# xs = odds scaled
xs = data['win_scaled']
xs.describe()

In [ ]:
# xr = win rank
xr = data['win_rank']
xr.describe()

In [ ]:
# xn = get num runners
xn = data['num_runners']
xn.describe()

In [ ]:
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

In [ ]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 1000
print('epochs = {}'.format(epochs))

tag = '{}30x30{}'.format(category, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=n))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [ ]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)